In [1]:
# 시트 이름 전체 확인
for ws in spreadsheet.worksheets():
    print(ws.title)


NameError: name 'spreadsheet' is not defined

In [ ]:
# ✅ Looker Studio용 컬럼 구성
looker_cols = ['관리본부', '관리지사', '계약번호', '상호', '시', '군', '구', '기타주소', 'Latitude', 'Longitude']
looker_df = filtered_df[looker_cols].copy()

# ✅ CSV로 저장 (한글 깨짐 방지 위해 utf-8-sig)
looker_csv_path = '/content/looker_location_data_full.csv'
looker_df.to_csv(looker_csv_path, index=False, encoding='utf-8-sig')

print("📁 Looker Studio용 CSV 파일 생성 완료: looker_location_data_full.csv")


In [ ]:
# 🚀 라이브러리 설치
!pip install --upgrade gspread gspread_dataframe oauth2client geopy

# 🚀 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 🚀 라이브러리 임포트
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import pandas as pd
from geopy.geocoders import Nominatim
from concurrent.futures import ThreadPoolExecutor
import time, re

# 🔑 인증 설정
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)

# 📂 구글 스프레드시트 열기
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit")
worksheet = spreadsheet.worksheet("DB_3.10")

# 📊 데이터프레임 가져오기 + 필요한 컬럼만 선택
raw_df = get_as_dataframe(worksheet, evaluate_formulas=True)
use_cols = ['설치주소', '관리본부', '관리지사', '계약번호', '상호']
df = raw_df[use_cols].dropna(subset=['설치주소']).reset_index(drop=True)

# 📍 주소 리스트 준비
address_list = df['설치주소'].tolist()

# 📌 위도/경도 추출 (병렬 처리 + 캐시)
geolocator = Nominatim(user_agent="geoapiExercises")
geo_cache = {}

def get_lat_lng_cached(address):
    if address in geo_cache:
        return geo_cache[address]
    try:
        location = geolocator.geocode(address)
        if location:
            result = (location.latitude, location.longitude)
        else:
            result = (None, None)
    except:
        result = (None, None)
    geo_cache[address] = result
    return result

with ThreadPoolExecutor(max_workers=5) as executor:
    latlng_results = list(executor.map(get_lat_lng_cached, address_list))

df['Latitude'], df['Longitude'] = zip(*latlng_results)

# 🧩 주소 파싱 함수 정의
def parse_korean_address(addr):
    addr = str(addr)
    시 = 군 = 구 = 나머지 = ''
    match_si = re.search(r'([가-힣]+시|[가-힣]+도)', addr)
    match_gun = re.search(r'([가-힣]+군)', addr)
    match_gu = re.search(r'([가-힣]+구)', addr)
    if match_si: 시 = match_si.group(1)
    if match_gun: 군 = match_gun.group(1)
    if match_gu: 구 = match_gu.group(1)
    시군구_끝 = max([m.end() for m in [match_si, match_gun, match_gu] if m], default=0)
    나머지 = addr[시군구_끝:].strip()
    return pd.Series([시, 군, 구, 나머지])

# 📌 시/군/구/기타주소 추출
df[['시', '군', '구', '기타주소']] = df['설치주소'].apply(parse_korean_address)

# 📌 최종 사용할 컬럼 구성
final_cols = ['설치주소', '관리본부', '관리지사', '계약번호', '상호', '시', '군', '구', '기타주소', 'Latitude', 'Longitude']
df_final = df[final_cols].copy()

# 📌 강북/강원 포함된 행만 필터링
filtered_df = df_final[df_final['관리본부'].fillna('').str.contains('강북|강원')].reset_index(drop=True)

# 📁 CSV 파일로 저장 (한글 깨짐 방지)
csv_path = '/content/looker_location_data_final.csv'
filtered_df.to_csv(csv_path, index=False, encoding='utf-8-sig')

print("✅ Looker Studio용 자동화 CSV 파일 생성 완료! → looker_location_data_final.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


✅ Looker Studio용 자동화 CSV 파일 생성 완료! → looker_location_data_final.csv


In [ ]:
# 📤 Google 스프레드시트에 테스트 결과 시트 저장
from gspread_dataframe import set_with_dataframe

# 테스트용 시트명
sheet_name = "DB_3.10_강북강원_10건"

# 시트 존재 시 삭제 후 재생성 (중복 방지)
try:
    spreadsheet.del_worksheet(spreadsheet.worksheet(sheet_name))
    print(f"🔄 기존 시트 '{sheet_name}' 삭제 완료")
except:
    pass

# 새 시트 생성 후 데이터 저장
test_sheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
set_with_dataframe(test_sheet, filtered_df)

print(f"✅ 스프레드시트에 '{sheet_name}' 시트 저장 완료!")


NameError: name 'spreadsheet' is not defined

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests
from concurrent.futures import ThreadPoolExecutor
import re

# ✅ 카카오 API 키
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# ✅ 구글 인증
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)

# ✅ 구글 시트 열기
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit")
worksheet = spreadsheet.worksheet("DB_3.10")

# ✅ 필요한 컬럼
cols_needed = [
    '설치주소', '상호', '계약번호', '관리지사', '관리본부',
    '서비스(소)', 'KTT월정료', '계약시작일', '계약종료일'
]
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[cols_needed].dropna(subset=["설치주소"]).copy()

# ✅ 주소 정제
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 지역 추출
def extract_region(addr):
    match = re.search(r"(서울|경기|부산|대구|인천|광주|대전|울산|세종|강원|충북|충남|전북|전남|경북|경남|제주)[\s]*(\S+?구|\S+?시|\S+?군)", addr)
    return f"{match.group(1)} {match.group(2)}" if match else None

# ✅ 위경도 요청
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            doc = res.json()['documents']
            if doc:
                loc = doc[0].get('road_address') or doc[0].get('address')
                if loc: return loc['y'], loc['x']
    except: pass
    # fallback: 원본 주소로
    try:
        fallback_url = f"https://dapi.kakao.com/v2/local/search/address.json?query={address}"
        res2 = requests.get(fallback_url, headers=headers)
        if res2.status_code == 200:
            doc2 = res2.json()['documents']
            if doc2:
                loc2 = doc2[0].get('road_address') or doc2[0].get('address')
                if loc2: return loc2['y'], loc2['x']
    except: pass
    return None, None

# ✅ 병렬 처리
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    lat_lng = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*lat_lng)
df["정제주소"] = df["설치주소"].apply(clean_address)
df["지역"] = df["정제주소"].apply(extract_region)

# ✅ 숫자화 및 평균 대체
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")

region_avg = df.dropna(subset=["위도", "경도"]).groupby("지역")[["위도", "경도"]].mean().reset_index()
region_avg.columns = ["지역", "대체위도", "대체경도"]
df = df.merge(region_avg, on="지역", how="left")

df["최종위도"] = df["위도"].combine_first(df["대체위도"])
df["최종경도"] = df["경도"].combine_first(df["대체경도"])

# ✅ 지도링크 + 지역차트용 좌표 컬럼
df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['최종위도']},{row['최종경도']}" if pd.notnull(row["최종위도"]) else "", axis=1)
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['최종위도']},{row['최종경도']}" if pd.notnull(row["최종위도"]) else "", axis=1)

# ✅ 저장
output_cols = cols_needed + ["최종위도", "최종경도", "위치좌표(위도,경도)", "지도링크"]
df_output = df[output_cols].copy()

# CSV 저장
csv_path = "/content/looker_map_combined_latlng.csv"
df_output.to_csv(csv_path, index=False, encoding='utf-8-sig')

print("✅ 완료: looker_map_combined_latlng.csv 파일 생성")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 완료: looker_map_combined_latlng.csv 파일 생성


In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests
from concurrent.futures import ThreadPoolExecutor
import re

# ✅ 카카오 API 키
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# ✅ 구글 인증
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)

# ✅ 구글 시트 열기
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit")
worksheet = spreadsheet.worksheet("DB_3.10")

# ✅ 필요한 컬럼
cols_needed = ['설치주소', '상호', '계약번호', '관리지사', '관리본부', '서비스(소)', 'KTT월정료', '계약시작일', '계약종료일']
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[cols_needed].dropna(subset=["설치주소"]).copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 지역 추출
def extract_region(addr):
    match = re.search(r"(서울|경기|부산|대구|인천|광주|대전|울산|세종|강원|충북|충남|전북|전남|경북|경남|제주)[\s]*(\S+?구|\S+?시|\S+?군)", addr)
    return f"{match.group(1)} {match.group(2)}" if match else None

# ✅ 카카오 위경도 요청 함수
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            doc = res.json()['documents']
            if doc:
                loc = doc[0].get('road_address') or doc[0].get('address')
                if loc: return loc['y'], loc['x']
    except: pass
    # fallback: 원본 주소로 재시도
    try:
        fallback_url = f"https://dapi.kakao.com/v2/local/search/address.json?query={address}"
        res2 = requests.get(fallback_url, headers=headers)
        if res2.status_code == 200:
            doc2 = res2.json()['documents']
            if doc2:
                loc2 = doc2[0].get('road_address') or doc2[0].get('address')
                if loc2: return loc2['y'], loc2['x']
    except: pass
    return None, None

# ✅ 병렬 처리
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    lat_lng = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*lat_lng)
df["정제주소"] = df["설치주소"].apply(clean_address)
df["지역"] = df["정제주소"].apply(extract_region)

# ✅ 숫자 변환 및 실패 대체 처리
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")

region_avg = df.dropna(subset=["위도", "경도"]).groupby("지역")[["위도", "경도"]].mean().reset_index()
region_avg.columns = ["지역", "대체위도", "대체경도"]
df = df.merge(region_avg, on="지역", how="left")

df["최종위도"] = df["위도"].combine_first(df["대체위도"])
df["최종경도"] = df["경도"].combine_first(df["대체경도"])

# ✅ Looker Studio용 추가 필드
df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['최종위도']},{row['최종경도']}" if pd.notnull(row["최종위도"]) else "", axis=1)
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['최종위도']},{row['최종경도']}" if pd.notnull(row["최종위도"]) else "", axis=1)

# ✅ 최종 저장
output_cols = cols_needed + ["최종위도", "최종경도", "위치좌표(위도,경도)", "지도링크"]
df_output = df[output_cols].copy()
csv_path = "/content/looker_map_combined_latlng.csv"
df_output.to_csv(csv_path, index=False, encoding='utf-8-sig')

print("✅ 완료: looker_map_combined_latlng.csv 생성")

# ✅ 성공률 검증
total = len(df_output)
success = df_output["최종위도"].notna().sum()
rate = round((success / total) * 100, 2)
print(f"총 {total}건 중 {success}건 성공 (성공률: {rate}%)")

Mounted at /content/drive
✅ 완료: looker_map_combined_latlng.csv 생성
총 4631건 중 4623건 성공 (성공률: 99.83%)


In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests, re
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

# ✅ 카카오 API 키
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"

# ✅ 인증 및 시트 열기
json_keyfile_path = "/content/drive/My Drive/Key/credentials.json"
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit")
worksheet = spreadsheet.worksheet("DB_3.10")

# ✅ 데이터 불러오기
cols_needed = ['설치주소', '상호', '계약번호', '관리지사', '관리본부', '구역담당', '서비스(소)', 'KTT월정료', '계약시작일', '계약종료일', '체납구분1', '체납구분2','정지일수 구간', '정지시작일자', '정지희망종료일', '최종수신일시']
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[cols_needed].dropna(subset=["설치주소"]).copy()

# ✅ 주소 정제 및 분해
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

def split_address(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    m1 = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    m2 = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    m3 = re.search(r'([가-힣]+[읍면동])', addr)
    if m1: 시 = m1.group(1)
    if m2: 군구 = m2.group(1)
    if m3: 읍면동 = m3.group(1)
    return pd.Series([시, 군구, 읍면동], index=["시", "군구", "읍면동"])

# ✅ 위도/경도 함수 (카카오 API)
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except: pass
    return None, None

# ✅ 병렬 처리로 위경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address)

# ✅ 좌표가 없는 행(제외 대상)
df_fail = df[df["위도"].isna() | df["경도"].isna()].copy()
df_success = df[df["위도"].notna() & df["경도"].notna()].copy()

# ✅ Looker용 필드 추가
df_success["최종위도"] = df_success["위도"]
df_success["최종경도"] = df_success["경도"]
df_success["지도링크"] = df_success.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}", axis=1)
df_success["위치좌표(위도,경도)"] = df_success.apply(lambda row: f"{row['위도']},{row['경도']}", axis=1)

# ✅ 최종 컬럼 정리
output_cols = cols_needed + ["시", "군구", "읍면동", "최종위도", "최종경도", "위치좌표(위도,경도)", "지도링크"]
df_success = df_success[output_cols].copy()

# ✅ 저장: CSV 파일
df_success.to_csv("/content/위도경도_성공.csv", index=False, encoding='utf-8-sig')
df_fail.to_csv("/content/위도경도_실패.csv", index=False, encoding='utf-8-sig')

# ✅ 스프레드시트에 시트 추가
now_str = datetime.now().strftime("위경도_%m%d%H%M")
try:
    spreadsheet.del_worksheet(spreadsheet.worksheet(now_str + "_성공"))
except: pass
ws_success = spreadsheet.add_worksheet(title=now_str + "_성공", rows="1000", cols="30")
set_with_dataframe(ws_success, df_success)

try:
    spreadsheet.del_worksheet(spreadsheet.worksheet(now_str + "_실패"))
except: pass
ws_fail = spreadsheet.add_worksheet(title=now_str + "_실패", rows="1000", cols="30")
set_with_dataframe(ws_fail, df_fail)

print(f"✅ 완료: 성공 {len(df_success)}건 / 실패 {len(df_fail)}건 저장 및 스프레드시트 반영됨")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 완료: 성공 4228건 / 실패 307건 저장 및 스프레드시트 반영됨


NameError: name 'df_output' is not defined

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests, re
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"  # ← 여기에 본인의 카카오 API 키 입력
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit"
SHEET_NAME = "DB_3.10"

# ✅ 구글 시트 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 필요한 컬럼만 선택
cols_needed = ['설치주소', '상호', '계약번호', '관리지사', '관리본부', '구역담당',
               '서비스(소)', 'KTT월정료', '계약시작일', '계약종료일',
               '체납구분1', '체납구분2', '정지일수 구간', '정지시작일자', '정지희망종료일', '최종수신일시']
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[cols_needed].dropna(subset=["설치주소"]).copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시/군구/읍면동 분리 함수
def split_address(addr):
    시 = 군구 = 읍면동 = ''
    if isinstance(addr, str):
        m1 = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
        m2 = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
        m3 = re.search(r'([가-힣]+[읍면동])', addr)
        if m1: 시 = m1.group(1)
        if m2: 군구 = m2.group(1)
        if m3: 읍면동 = m3.group(1)
    return pd.Series([시, 군구, 읍면동], index=["시", "군구", "읍면동"])

# ✅ 위도/경도 추출 함수
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address)

# ✅ 성공/실패 분리
df_success = df[df["위도"].notna() & df["경도"].notna()].copy()
df_fail = df[df["위도"].isna() | df["경도"].isna()].copy()

# ✅ Looker Studio용 필드 추가
df_success["최종위도"] = df_success["위도"]
df_success["최종경도"] = df_success["경도"]
df_success["위치좌표(위도,경도)"] = df_success.apply(lambda row: f"{row['위도']},{row['경도']}", axis=1)
df_success["지도링크"] = df_success.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}", axis=1)

# ✅ 최종 컬럼 구성
output_cols = cols_needed + ["시", "군구", "읍면동", "최종위도", "최종경도", "위치좌표(위도,경도)", "지도링크"]
df_success = df_success[output_cols].copy()

# ✅ 파일 저장
df_success.to_csv("/content/위도경도_성공.csv", index=False, encoding='utf-8-sig')
df_fail.to_csv("/content/위도경도_실패.csv", index=False, encoding='utf-8-sig')

# ✅ 스프레드시트 저장
now_str = datetime.now().strftime("위경도_%m%d%H%M")
for name, data in [(now_str + "_성공", df_success), (now_str + "_실패", df_fail[["설치주소"]])]:
    try:
        spreadsheet.del_worksheet(spreadsheet.worksheet(name))
    except:
        pass
    ws = spreadsheet.add_worksheet(title=name, rows="1000", cols="30")
    set_with_dataframe(ws, data)

# ✅ 성공률 출력
total = len(df)
success = len(df_success)
rate = round((success / total) * 100, 2)
print(f"✅ 총 {total}건 중 {success}건 성공 (성공률: {rate}%) → 스프레드시트 저장 완료")# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests, re
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"  # ← 여기에 본인의 카카오 API 키 입력
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit"
SHEET_NAME = "DB_0403"

# ✅ 구글 시트 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 필요한 컬럼만 선택
cols_needed = ['설치주소', '상호', '계약번호', '관리지사', '관리본부', '구역담당',
               '서비스(소)', 'KTT월정료', '계약시작일', '계약종료일',
               '체납구분1', '체납구분2', '정지일수 구간', '정지시작일자', '정지희망종료일', '최종수신일시']
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[cols_needed].dropna(subset=["설치주소"]).copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시/군구/읍면동 분리 함수
def split_address(addr):
    시 = 군구 = 읍면동 = ''
    if isinstance(addr, str):
        m1 = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
        m2 = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
        m3 = re.search(r'([가-힣]+[읍면동])', addr)
        if m1: 시 = m1.group(1)
        if m2: 군구 = m2.group(1)
        if m3: 읍면동 = m3.group(1)
    return pd.Series([시, 군구, 읍면동], index=["시", "군구", "읍면동"])

# ✅ 위도/경도 추출 함수
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address)

# ✅ 성공/실패 분리
df_success = df[df["위도"].notna() & df["경도"].notna()].copy()
df_fail = df[df["위도"].isna() | df["경도"].isna()].copy()

# ✅ Looker Studio용 필드 추가
df_success["최종위도"] = df_success["위도"]
df_success["최종경도"] = df_success["경도"]
df_success["위치좌표(위도,경도)"] = df_success.apply(lambda row: f"{row['위도']},{row['경도']}", axis=1)
df_success["지도링크"] = df_success.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}", axis=1)

# ✅ 최종 컬럼 구성
output_cols = cols_needed + ["시", "군구", "읍면동", "최종위도", "최종경도", "위치좌표(위도,경도)", "지도링크"]
df_success = df_success[output_cols].copy()

# ✅ 파일 저장
df_success.to_csv("/content/위도경도_성공.csv", index=False, encoding='utf-8-sig')
df_fail.to_csv("/content/위도경도_실패.csv", index=False, encoding='utf-8-sig')

# ✅ 스프레드시트 저장
now_str = datetime.now().strftime("위경도_%m%d%H%M")
for name, data in [(now_str + "_성공", df_success), (now_str + "_실패", df_fail[["설치주소"]])]:
    try:
        spreadsheet.del_worksheet(spreadsheet.worksheet(name))
    except:
        pass
    ws = spreadsheet.add_worksheet(title=name, rows="1000", cols="30")
    set_with_dataframe(ws, data)

# ✅ 성공률 출력
total = len(df)
success = len(df_success)
rate = round((success / total) * 100, 2)
print(f"✅ 총 {total}건 중 {success}건 성공 (성공률: {rate}%) → 스프레드시트 저장 완료")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests, re
from concurrent.futures import ThreadPoolExecutor

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit"
SHEET_NAME = "DB_0414"  # ← DB_3.10 으로 변경 가능

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 위도/경도 함수 (카카오 API)
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        return None, None
    return None, None

# ✅ 병렬 처리로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)
df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)

# ✅ 스프레드시트에 적용 (맨 오른쪽 컬럼 덧붙이기)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: DB_0403 시트에 위도/경도, 좌표, 지도링크 컬럼이 우측에 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 완료: DB_0403 시트에 위도/경도, 좌표, 지도링크 컬럼이 우측에 적용되었습니다.


In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit"
SHEET_NAME = "DB_0414"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시/군/구/읍/면/동 분리
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API 위도/경도 가져오기
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리 적용
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 지도 링크 + 좌표
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)
# df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)


# 아래 테스트해보고 정상여부 확인 필요
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("https://www.google.com/maps/dir/?api=1&destination={row["위도"]},{row["경도"]}", "길찾기")'
    if pd.notnull(row['위도']) else "", axis=1
)



# ✅ 원본 시트 맨 오른쪽에 덧붙여 저장
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동 + 위도/경도 + 지도링크가 DB_0413 시트 맨 오른쪽에 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [ ]:
!pip install --upgrade gspread gspread_dataframe oauth2client requests

In [ ]:
!pip install --upgrade gspread gspread_dataframe oauth2client requests

In [ ]:
!pip install --upgrade gspread gspread_dataframe oauth2client requests

In [ ]:
# ✅ 구글 드라이브 인증
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리 불러오기
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit"

# ✅ 인증 및 스프레드시트 연결
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)


Mounted at /content/drive


In [ ]:
# 중복찾기

# ✅ DB_0403 시트 열기
worksheet = spreadsheet.worksheet("DB_0403")
df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how='all').fillna("")

# ✅ 전체 컬럼 값이 완전히 동일한 행만 찾기
duplicate_rows = df[df.duplicated(keep=False)]  # 중복 포함 전체 가져오기
grouped = duplicate_rows.groupby(duplicate_rows.columns.tolist()).filter(lambda x: len(x) > 1)

# ✅ 중복 행만 새로운 시트로 저장
same_rows_sheet = "DB_0403_동일행만"
try:
    spreadsheet.del_worksheet(spreadsheet.worksheet(same_rows_sheet))
except:
    pass
ws_same = spreadsheet.add_worksheet(title=same_rows_sheet, rows="1000", cols="30")
set_with_dataframe(ws_same, grouped)


print(f"✅ 모든 셀 값이 동일한 {len(grouped)}건을 '{same_rows_sheet}' 시트에 저장 완료")


✅ 모든 셀 값이 동일한 38건을 'DB_0403_동일행만' 시트에 저장 완료


In [ ]:
# 중복제거

# ✅ 기존 df에서 동일한 행 제거
df_no_duplicates = df.drop_duplicates(keep=False)

# ✅ DB_0403 시트 덮어쓰기
set_with_dataframe(worksheet, df_no_duplicates, include_index=False, include_column_header=True, resize=True)

print(f"✅ 동일한 전체 행 {len(grouped)}건 제거 후 {len(df_no_duplicates)}건만 DB_0403 시트에 반영 완료")

✅ 동일한 전체 행 38건 제거 후 5623건만 DB_0403 시트에 반영 완료


In [ ]:
# 관리고객 해지고ARPU 위경도 적용

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1RsLDDjZGbjPkxXEdVoKhynksFAXO3kTEnXjphXOTKL0/edit?usp=sharing"
SHEET_NAME = "관리고객 4월(4.9_강다솜)_보고서용"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시/군/구/읍/면/동 분리
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API 위도/경도 가져오기
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리 적용
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 지도 링크 + 좌표
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)
# df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)


# 아래 테스트해보고 정상여부 확인 필요
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("https://www.google.com/maps/dir/?api=1&destination={row["위도"]},{row["경도"]}", "길찾기")'
    if pd.notnull(row['위도']) else "", axis=1
)



# ✅ 원본 시트 맨 오른쪽에 덧붙여 저장
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동 + 위도/경도 + 지도링크가 DB_0414 시트 맨 오른쪽에 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


WorksheetNotFound: DB_0414

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1RsLDDjZGbjPkxXEdVoKhynksFAXO3kTEnXjphXOTKL0/edit?usp=sharing"
SHEET_NAME = "DB_0414"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시/군/구/읍/면/동 분리
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API 위도/경도 가져오기
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리 적용
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 지도 링크 + 좌표
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)
df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)






# ✅ 원본 시트 맨 오른쪽에 덧붙여 저장
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동 + 위도/경도 + 지도링크가 DB_0414 시트 맨 오른쪽에 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


WorksheetNotFound: DB_0414

In [ ]:
# 3월말 유지시설 설치주소 위경도 적용

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1pjj0r5qmmGUtl9zA2ekT14xgqsOXuR0zFjSDitWhHmA/edit?usp=sharing"
SHEET_NAME = "카카오지도"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시/군/구/읍/면/동 분리
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API 위도/경도 가져오기
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리 적용
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 지도 링크 + 좌표
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)
df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)






# ✅ 원본 시트 맨 오른쪽에 덧붙여 저장
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동 + 위도/경도 + 지도링크가 DB_0403 시트 맨 오른쪽에 적용되었습니다.")

Mounted at /content/drive


WorksheetNotFound: 카카오지도

In [ ]:
# ✅ 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import requests, re
from concurrent.futures import ThreadPoolExecutor

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1q3-bk4NOgk4j4jhpWm6gZX6HSSue8Mct4TGF2cualVI/edit?usp=sharing"
SHEET_NAME = "해지DB250101-0415"  # ← DB_3.10 으로 변경 가능

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 위도/경도 함수 (카카오 API)
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        return None, None
    return None, None

# ✅ 병렬 처리로 위도/경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)
df["위치좌표(위도,경도)"] = df.apply(lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)
df["지도링크"] = df.apply(lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1)

# ✅ 스프레드시트에 적용 (맨 오른쪽 컬럼 덧붙이기)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: DB_0403 시트에 위도/경도, 좌표, 지도링크 컬럼이 우측에 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 완료: DB_0403 시트에 위도/경도, 좌표, 지도링크 컬럼이 우측에 적용되었습니다.


In [ ]:
## 해지시설 위경도 적용 250418

In [ ]:
# ✅ 필수 패키지 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리 불러오기
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"  # 여기에 본인 API 키 입력
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1HVKhkyki9h3bOdI9RPhZsvQh3pjEcexdl1ISZiXX1lk/edit"  # ← 해지 스프레드시트
SHEET_NAME = "해지DB250101-0415"

# ✅ 인증 및 구글 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 가져오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 주소에서 시군구, 읍면동 추출
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 위도/경도 추출 함수 (카카오 API)
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리로 위경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구, 읍면동 분리
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 지도 링크 및 좌표 컬럼
df["위치좌표(위도,경도)"] = df.apply(
    lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크_URL"] = df.apply(
    lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("{row["지도링크_URL"]}", "길찾기")' if row["지도링크_URL"] else "", axis=1
)

# ✅ 결과 저장 (시트 맨 오른쪽에 추가)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다."))

# ✅ 시트에 덧붙이기 (맨 오른쪽)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 해지 시트에 시군구, 읍면동 + 위도/경도 + 길찾기 링크가 적용되었습니다.")

SyntaxError: unmatched ')' (<ipython-input-4-7212187d62e6>, line 96)

In [ ]:
zz

In [ ]:
# ✅ 패키지 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1pjj0r5qmmGUtl9zA2ekT14xgqsOXuR0zFjSDitWhHmA/edit?usp=sharing"
SHEET_NAME = "해지DB250101-0415"

# ✅ 구글 시트 인증 및 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터프레임 불러오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 주소에서 시군구, 읍면동 분리
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API로 위도/경도 가져오기
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리로 위경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구, 읍면동 분리
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 지도 링크 및 좌표 컬럼
df["위치좌표(위도,경도)"] = df.apply(
    lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크_URL"] = df.apply(
    lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("{row["지도링크_URL"]}", "길찾기")' if row["지도링크_URL"] else "", axis=1
)

# ✅ 결과 저장 (시트 맨 오른쪽에 추가)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


WorksheetNotFound: 해지DB250101-0415

In [ ]:
# ✅ 필수 패키지 설치
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리 불러오기
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/13hT-Ea5-P5Vi-37d1dZp2ev_Mi3QUQ5Rh9XT33sqxhA/edit?usp=sharing"
SHEET_NAME = "DB_0403"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 가져오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시군구, 읍면동 분리 함수
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API를 통한 위도/경도 추출
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리 위경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 좌표, 지도링크 URL, 길찾기 하이퍼링크 추가
df["위치좌표(위도,경도)"] = df.apply(
    lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크_URL"] = df.apply(
    lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("{row["지도링크_URL"]}", "길찾기")' if row["지도링크_URL"] else "", axis=1
)

# ✅ 결과 저장 (시트 맨 오른쪽에 추가)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.")

Mounted at /content/drive
✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.


In [ ]:
영업기

In [ ]:
https://docs.google.com/spreadsheets/d/1Q1310s9-TCNmv_LFs9JKoSmhoeziKFH0o4CzdCfi1iM/edit

In [ ]:
# ✅ 필수 패키지 설치__인허가  영업기회
!pip install --upgrade gspread gspread_dataframe oauth2client requests

# ✅ 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# ✅ 라이브러리 불러오기
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# ✅ 설정
KAKAO_API_KEY = "af04a0a8e5416c95eaa04cccc060031d"
CREDENTIALS_PATH = "/content/drive/My Drive/Key/credentials.json"
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1Q1310s9-TCNmv_LFs9JKoSmhoeziKFH0o4CzdCfi1iM/edit"
SHEET_NAME = "20250423_전체_최종결과물"

# ✅ 인증 및 시트 열기
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_PATH, scope)
gc = gspread.authorize(credentials)
spreadsheet = gc.open_by_url(SPREADSHEET_URL)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ 데이터 가져오기
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df[df["설치주소"].notna()].copy()

# ✅ 주소 정제 함수
def clean_address(addr):
    addr = str(addr)
    addr = re.sub(r'\(.*?\)', '', addr)
    addr = re.sub(r'\d+층|\d+호', '', addr)
    addr = re.sub(r'[^\w\s가-힣\d-]', '', addr)
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr

# ✅ 시군구, 읍면동 분리 함수
def split_address_parts(addr):
    addr = str(addr)
    시 = 군구 = 읍면동 = ''
    match_si = re.search(r'(서울|부산|대구|인천|광주|대전|울산|세종|제주|경기|강원|충북|충남|전북|전남|경북|경남)', addr)
    match_gungu = re.search(r'([가-힣]+구|[가-힣]+시|[가-힣]+군)', addr)
    match_eupmyun = re.search(r'([가-힣]+[읍면동])', addr)
    if match_si: 시 = match_si.group(1)
    if match_gungu: 군구 = match_gungu.group(1)
    if match_eupmyun: 읍면동 = match_eupmyun.group(1)
    return pd.Series([시, 군구, 읍면동])

# ✅ 카카오 API를 통한 위도/경도 추출
def get_lat_lng_kakao(address):
    cleaned = clean_address(address)
    url = f"https://dapi.kakao.com/v2/local/search/address.json?query={cleaned}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    try:
        res = requests.get(url, headers=headers, timeout=3)
        if res.status_code == 200:
            docs = res.json().get("documents")
            if docs:
                loc = docs[0].get("road_address") or docs[0].get("address")
                if loc:
                    return float(loc["y"]), float(loc["x"])
    except:
        pass
    return None, None

# ✅ 병렬 처리 위경도 추출
addresses = df["설치주소"].tolist()
with ThreadPoolExecutor(max_workers=10) as executor:
    latlng_list = list(executor.map(get_lat_lng_kakao, addresses))

df["위도"], df["경도"] = zip(*latlng_list)

# ✅ 시군구 읍면동 분리
df[["시", "군구", "읍면동"]] = df["설치주소"].apply(split_address_parts)

# ✅ 좌표, 지도링크 URL, 길찾기 하이퍼링크 추가
df["위치좌표(위도,경도)"] = df.apply(
    lambda row: f"{row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크_URL"] = df.apply(
    lambda row: f"https://www.google.com/maps?q={row['위도']},{row['경도']}" if pd.notnull(row['위도']) else "", axis=1
)
df["지도링크"] = df.apply(
    lambda row: f'=HYPERLINK("{row["지도링크_URL"]}", "길찾기")' if row["지도링크_URL"] else "", axis=1
)

# ✅ 결과 저장 (시트 맨 오른쪽에 추가)
set_with_dataframe(worksheet, df, include_index=False, include_column_header=True, resize=False)

print("✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 완료: 시군구, 읍면동, 위도/경도, 지도링크_URL, 길찾기 링크가 모두 적용되었습니다.
